<a href="https://colab.research.google.com/github/nathanfleight/scripts/blob/main/Copy_of_docker101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Docker, NVIDIA Docker, dan Tensorflow Docker Image.

**Eksperimen!**

1. Install Docker (https://docs.docker.com/engine/install/ubuntu/) - pake yg repository. 
2. Jangan lupa lakuin https://docs.docker.com/engine/install/linux-postinstall/ (sampe kamu bisa 'docker hello-world' alias bisa njalanin command docker tanpa perlu sudo). Habis itu log out - log in lagi linux nya. 
3. Install NVIDIA Docker https://github.com/NVIDIA/nvidia-docker : sampe bisa nvidia-smi. 
4. Install Tensorflow Docker Image. 
Ini nginstall Tensorflow 2.1.1 pake GPU & jupyter
```
docker pull tensorflow/tensorflow:2.1.1-gpu-jupyter
```
5. Habis itu coba jalanin docker run. Tp yg ini error. 
```
docker run --rm --runtime=nvidia -v -it -p 8888:8888 tensorflow/tensorflow:2.1.1-gpu-jupyter
```
Solusi nemu di https://github.com/NVIDIA/nvidia-docker/issues/838 . 
Solusi pertama yg dicoba: 
```
sudo systemctl daemon-reload
sudo systemctl restart docker
```
Tapi masih error, terus coba ganti sintaks nya ada di: 
```
docker run --rm --gpus all -v -it -p 8888:8888 tensorflow/tensorflow:2.1.1-gpu-jupyter
```
ganti '--runtime=nvidia' jadi '--gpus all'. 

6. Harusnya udah bisa launch notebook, cek 'nvidia-smi' sama 'nvcc --version' di jupyter notebook udah bisa nunjukin output yg seharusnya.  


# Analisis Sintaks Docker

Analisis sintaks docker (dan cara mengakses dan menyimpan file jupyter notebook) https://docs.docker.com/engine/reference/commandline/run/: 
```
docker run --rm --gpus all -it -p 8888:8888 --name cekDir12 -v $PWD/Documents:/tf tensorflow/tensorflow:2.1.1-gpu-jupyter
```

*   --gpus all > set pake seluruh gpu. 
*   -it > menginstruksi Docker untuk mengalokasi pseudo-TTY yg terhubung dg container's stdin, menciptakan interactive bash shell di container. 
*   -p > port yg dipakai oleh si jupyter notebook (dlm kasus ini localhost). 
*   -name > nama dari container, agar suatu saat kalo mau njalanin container ini lagi, cukup pake nama ini (contoh di bawah). 
*   -v > sintaks yg memfasilitasi komunikasi antara host dan container. Jadi kalau misal ini tidak diset, secara otomatis seluruh file .ipynb akan disimpan di folder container '/tf' (tp masih belum ketemu ini ada dimana, dan sepertinya tidak penting krn file .ipynb yg kita ubah sudah tersimpan di direktori host kita). Kalau pengen nge-mount ke direktori kita sendiri, bisa pake '-v /host/directory:/container/directory', maka sesuai dicontoh: '-v -dollar-PWD/Documents:/tf', maka jupyter notebook akan dimount di -dollar-PWD (direktori home dari usermu), di dalam direktori Documents. 
*   tensorflow/tensorflow:2.1.1-gpu-jupyter > ini adalah image yg di mount ke dalam container. 

Bonus : --rm artinya setelah container ditutup, selesai dijalankan, maka container akan dihapus secara otomatis. Sebaiknya ini dilakukan krn tidak masalah jika container dihapus, **krn data yg kita punya sudah tersimpan di direktori host yg kita masukkan**. 


# Start & Stop Docker Container

Kalau kita nge-set nama dari container yg dibuat, maka kita bisa menjalankan container itu lagi kapan saja. Tp ini tidak berlaku (?) untuk jupyter notebook, krn kita butuh token utk masuk (maka dari itu sebaiknya utk jupyter notebook container, sekali pakai saja). 

Untuk nge-start: 
```
docker start -nama_container-
```
Untuk nge-stop: 
```
docker stop -nama_container-
```


# Install Additional PIP Package

Cara nambahin **additional package** di Tensorflow image (atau image apapun).

Sepertinya, defaultnya tensorflow docker itu dia cuma punya bbrp default package: Pillow, h5py, keras_preprocessing, matplotlib, mock, numpy, scipy, sklearn, pandas, portpicker, enum34. 

Kadang kita butuh package lain, maka kita perlu membuat sebuah image baru yg mencakup package yg kita butuhkan. 

Kita tidak perlu built semuanya dari 0 (CUDA, CuDNN, tensorflow), cukup pakai tensorflow image yg sudah kita download sbg basis nya. 

Untuk itu, kita cukup membuat Dockerfile baru, yg isinya sbg berikut: 
```
FROM tensorflow/tensorflow:latest-gpu

RUN pip install imageio
RUN pip install opencv-python 
```
Simpan di file yg diberi nama 'Dockerfile'. 

Selanjutnya, file ini perlu di built dulu sebelum bisa dipakai. Caranya adalah, masuk ke dalam direktori dimana Dockerfile disimpan, lalu pd terminal:
```
docker image build . --tag locak:tensorflow-extra
```
Catatan: --tag adalah nama dari image yg dibuat. 'locak' menunjukkan nama repository, sedangkan 'tensorflow-extra' adalah tag. 

Jika image sudah berhasil di-build, maka untuk menggunakan image tsb:
```
docker run --gpus all -it --rm -v $PWD:/tmp -w /tmp locak:tensorflow-extra python cek_tf.py
```

Catatan: untuk melihat image apa saja yg ada pd docker:
```
docker image ls
```
Referensi: https://stackoverflow.com/questions/58191215/how-to-add-python-libraries-to-docker-image .

Alternatif lain (install di container pd saat run, tp belum dicek): https://stackoverflow.com/questions/40564423/tensorflow-docker-container-on-windows-how-to-install-python-package .

# Coretan Log
Abaikan saja. 



1.   Install docker: 

https://docs.docker.com/get-started/ (official). 

https://docs.docker.com/engine/install/linux-postinstall/ (post install ? cek bisa gak tanpa sudo). 

https://docs.docker.com/engine/reference/run/ (reference buat sintaks run nya). 
2.   Install NVIDIA Driver dulu, tapi tidak perlu install CUDA Toolkit (ini langsung keinstall pd saat install NVIDIA Docker):

3.   Install NVIDIA Docker: https://github.com/NVIDIA/nvidia-docker (official). 

daftar tensorflow image: https://hub.docker.com/r/tensorflow/tensorflow/tags .

https://arnon.dk/installing-nvidia-docker/ (referensi install NVIDIA Docker). 

https://cnvrg.io/how-to-setup-docker-and-nvidia-docker-2-0-on-ubuntu-18-04/ (referensi install NVIDIA Docker, tp kayaknya udah lama). 

https://medium.com/@sh.tsang/docker-tutorial-5-nvidia-docker-2-0-installation-in-ubuntu-18-04-cb80f17cac65 (referens install NVIDIA Docker, tp kayaknya udah lama). 




Docker & Python

run versi terbaru python di docker: https://realpython.com/python-versions-docker/

run python file di docker : https://stackoverflow.com/questions/47202705/how-to-run-my-python-script-on-docker



Docker & Tensorflow

install TF Docker Ubuntu 18.04 with GPU: https://medium.com/@madmenhitbooker/install-tensorflow-docker-on-ubuntu-18-04-with-gpu-support-ed58046a2a56

setup TF-GPU, Anaconda: https://blog.rubell.com/how-to-install-anaconda-tensorflow-2-gpu-in-docker-on-ubuntu/

setup TF Python3 environment: https://winsmarts.com/easiest-way-to-setup-a-tensorflow-python3-environment-with-docker-5fc3ec0f6df1

docker TF2 (pake jupyter notebook) : https://analyticsindiamag.com/docker-solution-for-tensorflow-2-0-how-to-get-started/

referensi lain: https://missinglink.ai/guides/tensorflow/tensorflow-docker/


Docker & Cloud Platform 

Jupyter + Tensorflow + GPU + Docker + Google Compute Engine: https://medium.com/google-cloud/jupyter-tensorflow-nvidia-gpu-docker-google-compute-engine-4a146f085f17

# Lainnya: Kubernetes (link)

Kubernetes adalah platform untuk orkestrasi container. https://kubernetes.io/id/docs/concepts/overview/what-is-kubernetes/

Perbedaan antara kubernetes vs docker: https://azure.microsoft.com/en-us/topic/kubernetes-vs-docker/ 

Dari artikel tsb akan jelas bahwa padanan utk kubernetes adalah docker swarm. 

Dan ada lagi saingannya yaitu Apache Mesos: https://vexxhost.com/blog/kubernetes-mesos-comparison-containerization/ .